外部資料

In [57]:
import pandas as pd
import numpy as np
from glob import glob, iglob

df = pd.DataFrame()

# iglob使用較少memory
for csv in iglob('../0821/*_top500_clean.csv'):
    tmp_df = pd.read_csv(csv)
    df = df.append(tmp_df, ignore_index=True)

內部資料

In [2]:
df_gps = pd.read_csv("F:\\NCTU\\lab\\奧丁丁\\奧丁丁資料前處理\\OwlTing_整合資料\\csv\\data_gps.csv")

總共1500多家飯店

In [3]:
df_gps.latitude.isna().sum()

73

緯度為null

In [7]:
df_gps.longitude.isna().sum()

232

經度為0

In [9]:
len(df_gps.loc[df_gps.latitude == 0])

364

In [11]:
len(df_gps.loc[df_gps.longitude == 0])

200

移除經緯度為0或是null的飯店

In [15]:
df_gps_clean = df_gps.dropna(subset=['latitude', 'longitude']) # 僅刪除經緯度有0的

In [17]:
df_gps_clean = df_gps_clean[df_gps_clean.latitude!=0]
df_gps_clean = df_gps_clean[df_gps_clean.longitude!=0]

In [18]:
df_gps_clean.isna().sum()

hotel_id       0
country      196
city         339
latitude       0
longitude      0
dtype: int64

In [58]:
def get_first(x, length):
    return x[:length]

# 把經緯度成字串保留想要的位數
df['lat'] = df['lat'].astype('str').apply(get_first, args=(8,)).astype('float')
df['lng'] = df['lng'].astype('str').apply(get_first, args=(10,)).astype('float')

內外部資料都只用小數點4位看能不能和

In [63]:
def get_first(x, length):
    return x[:length]

# 把經緯度成字串保留想要的位數
df['lat'] = df['lat'].astype('str').apply(get_first, args=(7,)).astype('float')
df['lng'] = df['lng'].astype('str').apply(get_first, args=(8,)).astype('float')
df_gps_clean['latitude'] = df_gps_clean['latitude'].astype('str').apply(get_first, args=(7,)).astype('float')
df_gps_clean['longitude'] = df_gps_clean['longitude'].astype('str').apply(get_first, args=(8,)).astype('float')

找有沒有經緯度完全重複的

In [59]:
df.duplicated(subset=['lat', 'lng'], keep=False).sum()

637

In [47]:
df_gps_clean.duplicated(subset=['latitude', 'longitude'], keep=False).sum()

39

經緯度同時重疊

In [60]:
df[df.duplicated(subset=['lat', 'lng'], keep=False)]

,title,uri,lat,lng,hotel_address,hotel_star,hotel_city,hotel_section,price_range,avg_rating,comment_count
45,香緹汽車旅館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.48041,120.463180,西區八德路2號,3.0,嘉義,西區,"NT$1,980 - NT$2,922",3.5,4.0
70,波士頓大飯店,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.48041,120.463180,東區中正路673號,3.0,嘉義,西區,"NT$691 - NT$1,508",3.0,11.0
72,雙鳳大旅社,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.99303,121.593353,復興街5號,2.0,花蓮,花蓮市,"NT$597 - NT$2,766",3.5,11.0
73,小莫里民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.03164,121.604553,新城鄉北埔路159號,3.5,花蓮,新城,"NT$1,540 - NT$3,238",5.0,3.0
77,旅路渡假商務旅館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.97014,121.605148,重慶路255號,3.0,花蓮,花蓮市,"NT$1,697 - NT$2,703",3.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...
4008,宜蘭民宿-幸福蝸牛,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.85802,121.828201,五結鄉利澤西路6號,3.0,宜蘭,五結,"NT$1,918 - NT$4,621",3.5,3.0
4026,奥聖思民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.70161,121.797393,礁溪鄉奇立丹路76巷2－5號 遵循ONSENSE標誌。,4.0,宜蘭,礁溪,"NT$3,206 - NT$8,487",4.0,12.0
4028,田野小徑,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.80044,121.756149,礁溪鄉份尾一路3號,3.5,宜蘭,礁溪,"NT$2,075 - NT$3,395",4.0,1.0
4033,賓成精品商旅,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.67275,121.809745,西後街26-1號,3.0,宜蘭,宜蘭市,"NT$1,163 - NT$2,389",3.0,9.0


經度重疊

In [32]:
df[df['lat'].isin(df['lat'][df['lat'].duplicated()])]

,title,uri,lat,lng,hotel_address,hotel_star,hotel_city,hotel_section,price_range,avg_rating,comment_count
3,嘉義優遊商旅,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.48274,120.463,西區中山路617號11樓,3.0,嘉義,東區,"NT$1,163 - NT$2,734",3.5,41.0
39,楓雅SPA 汽車旅館 (大雅館),https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.48274,120.431,大雅路二段256號,3.5,嘉義,東區,"NT$1,194 - NT$1,697",4.0,1.0
45,香緹汽車旅館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.48041,120.463,西區八德路2號,3.0,嘉義,西區,"NT$1,980 - NT$2,922",3.5,4.0
70,波士頓大飯店,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.48041,120.463,東區中正路673號,3.0,嘉義,西區,"NT$691 - NT$1,508",3.0,11.0
72,雙鳳大旅社,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.99303,121.593,復興街5號,2.0,花蓮,花蓮市,"NT$597 - NT$2,766",3.5,11.0
...,...,...,...,...,...,...,...,...,...,...,...
4028,田野小徑,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.80044,121.756,礁溪鄉份尾一路3號,3.5,宜蘭,礁溪,"NT$2,075 - NT$3,395",4.0,1.0
4029,玫瑰莊園民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.67588,121.751,冬山鄉梅花路742號,3.5,宜蘭,冬山,"NT$1,980 - NT$4,527",0.0,0.0
4030,夜市橙堡渡假旅館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.82802,121.774,新民路19號,3.5,宜蘭,宜蘭市,"NT$1,320 - NT$2,515",4.0,12.0
4033,賓成精品商旅,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.67275,121.809,西後街26-1號,3.0,宜蘭,宜蘭市,"NT$1,163 - NT$2,389",3.0,9.0


嘗試能不能concat

In [65]:
pd.merge(df, df_gps_clean, left_on=['lat', 'lng'], right_on=['latitude', 'longitude']).head()

,title,uri,lat,lng,hotel_address,hotel_star,hotel_city,hotel_section,price_range,avg_rating,comment_count,hotel_id,country,city,latitude,longitude
0,船長不在家旅舍,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.9903,121.6051,國聯3路85號,2.0,花蓮,花蓮市,"NT$786 - NT$2,578",4.5,3.0,795,TW,NaN,23.9903,121.6051
1,享海民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.9717,121.5976,中正路169巷1-7號,3.5,花蓮,花蓮市,-NT$1,5.0,14.0,962,台灣,花蓮,23.9717,121.5976
2,漫漫日出,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.9806,121.6091,花蓮市海濱街75號,3.0,花蓮,花蓮市,"NT$2,641 - NT$5,784",5.0,22.0,238,台灣,花蓮,23.9806,121.6091
3,輕停文旅,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.0053,121.6223,花蓮市國興二街92號,3.0,花蓮,花蓮市,"NT$503 - NT$2,483",3.0,1.0,233,台灣,花蓮,24.0053,121.6223
4,幸福加油站民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.9762,121.6026,信義街93號,3.0,花蓮,花蓮市,"NT$1,320 - NT$3,301",5.0,4.0,256,TW,花蓮市,23.9762,121.6026


能和的飯店數目

In [66]:
len(pd.merge(df, df_gps_clean, left_on=['lat', 'lng'], right_on=['latitude', 'longitude']))

68